In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from tensorflow.keras import layers, models
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
import tensorflow as tf
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt

/home/nwertheim/miniconda3/envs/myenv/bin/python


ImportError: /lib64/libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/nwertheim/miniconda3/envs/myenv/lib/python3.8/site-packages/scipy/optimize/_highs/_highs_wrapper.cpython-38-x86_64-linux-gnu.so)

In [3]:
data_file = os.path.join(univariate_data_path, 'merged_univariate.npy')
data = np.load(data_file, allow_pickle=True)
print(data.shape)

# Check for NaN or Inf values in signals
for sample in data:
    if np.isnan(sample['signal']).any() or np.isinf(sample['signal']).any():
        print(f"NaN or Inf detected in {sample['record_name']}")

(666,)


In [8]:
# Masking function using patch-based masking
def mask_data(x, mask_ratio=0.5, patch_size=8):
    """ Apply patch-based masking to 1D signals """
    x_masked = np.copy(x)
    num_patches = x.shape[1] // patch_size  # Number of patches
    mask = np.random.rand(num_patches) < mask_ratio  # Randomly mask patches
    for i in range(num_patches):
        if mask[i]:
            x_masked[:, i * patch_size:(i + 1) * patch_size, :] = 0  # Zero out patches
    return x_masked

# Create windows from the data
def create_windows(sequence, window_size, step_size):
    windows = [sequence[i:i+window_size] for i in range(0, len(sequence) - window_size + 1, step_size)]
    return np.array(windows)

# Set windowing parameters
window_size = 500  
step_size = 250    

# Assuming `data` is a list of dictionaries with signal data
# Apply windowing to all records
all_windows = [create_windows(record['signal'], window_size, step_size) for record in data]
all_windows = np.concatenate(all_windows, axis=0)

# Reshape for Conv1D (batch_size, time_steps, channels)
all_windows = np.expand_dims(all_windows, axis=-1)  
print(f"Processed window shape: {all_windows.shape}")

Processed window shape: (454061, 500, 1)


In [12]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers, models

# class FullyConvolutionalMaskedAutoencoder:
#     def __init__(self, input_shape, num_layers=3, filters=[64, 32, 16]):
#         self.input_shape = input_shape
#         self.num_layers = num_layers
#         self.filters = filters
#         self.model = self.build_model()

#     def build_model(self):
#         model = models.Sequential()
        
#         # Encoder
#         for i in range(self.num_layers):
#             model.add(layers.Conv1D(filters=self.filters[i], kernel_size=3, activation='relu', padding='same', input_shape=self.input_shape if i == 0 else None))
#             if i < self.num_layers - 1:  # No pooling on the last layer
#                 model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
        
#         # Decoder
#         for i in range(self.num_layers):
#             model.add(layers.Conv1DTranspose(filters=self.filters[-(i + 1)], kernel_size=3, activation='relu', padding='same'))
#             if i < self.num_layers - 1:  # No upsampling on the last layer
#                 model.add(layers.UpSampling1D(size=2))
        
#         # Final output layer
#         model.add(layers.Conv1DTranspose(filters=1, kernel_size=3, activation='sigmoid', padding='same'))  # Output layer
        
#         return model

#     def compile_model(self, optimizer='adam', loss='mean_squared_error'):
#         self.model.compile(optimizer=optimizer, loss=loss)

#     def fit(self, X_train, epochs=50, batch_size=32, validation_split=0.2):
#         self.model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

# # Example usage
# window_size = 500  # Your window size
# input_shape = (window_size, 1)  # Shape of each window
# autoencoder = FullyConvolutionalMaskedAutoencoder(input_shape, num_layers=3, filters=[64, 32, 16])

# # Compile the model
# autoencoder.compile_model()

# # Assuming your data is in `all_windows`
# # all_windows should be shaped as (454061, 500, 1)
# autoencoder.fit(all_windows, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/10, Train Loss: 8.804382107138549, Val Loss: 8.608450144928387
Epoch 2/10, Train Loss: 8.793140871590758, Val Loss: 8.608437879645043
Epoch 3/10, Train Loss: 8.793221434629492, Val Loss: 8.608435525114558
Epoch 4/10, Train Loss: 8.793335143888896, Val Loss: 8.608434715432294
Epoch 5/10, Train Loss: 8.793316783736591, Val Loss: 8.608434350747638
Epoch 6/10, Train Loss: 8.793219392319815, Val Loss: 8.60843413610371
Epoch 7/10, Train Loss: 8.793383032566565, Val Loss: 8.608433981526478
Epoch 8/10, Train Loss: 8.79317007357172, Val Loss: 8.608433697742841
Epoch 9/10, Train Loss: 8.793352258896391, Val Loss: 8.60843380376266
Epoch 10/10, Train Loss: 8.793260908803106, Val Loss: 8.608433591050947


AttributeError: 'FullyConvolutionalMaskedAutoencoder' object has no attribute 'extract_encoder'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

class FullyConvolutionalMaskedAutoencoder:
    def __init__(self, input_shape, num_layers=3, filters=[64, 32, 16]):
        self.input_shape = input_shape
        self.num_layers = num_layers
        self.filters = filters
        self.model = self.build_model()

    def build_model(self):
        # Create input layer explicitly
        inputs = layers.Input(shape=self.input_shape)

        # Encoder
        x = inputs
        for i in range(self.num_layers):
            x = layers.Conv1D(filters=self.filters[i], kernel_size=3, activation='relu', padding='same')(x)
            if i < self.num_layers - 1:  
                x = layers.MaxPooling1D(pool_size=2, padding='same')(x)  # Downsampling

        # Decoder
        for i in range(self.num_layers):
            x = layers.Conv1DTranspose(filters=self.filters[-(i + 1)], kernel_size=3, activation='relu', padding='same')(x)
            if i < self.num_layers - 1:  
                x = layers.UpSampling1D(size=2)(x)  # Upsampling

        # Final output layer
        outputs = layers.Conv1DTranspose(filters=1, kernel_size=3, activation='sigmoid', padding='same')(x)

        # **Fix the output shape using Cropping1D**
        crop_size = (outputs.shape[1] - self.input_shape[0]) // 2
        if crop_size > 0:
            outputs = layers.Cropping1D((crop_size, crop_size))(outputs)  # Ensure it's a tuple (left, right)

        # Create the final model
        model = models.Model(inputs=inputs, outputs=outputs)

        return model
        
    def compile_model(self, optimizer='adam', loss='mean_squared_error'):
        self.model.compile(optimizer=optimizer, loss=loss)

    def fit(self, X_train, epochs=50, batch_size=32, validation_data=None):
        if validation_data is not None:
            val_data, val_labels = validation_data
            val_loss = []

            # Train the model and track validation loss manually
            for epoch in range(epochs):
                print(f"Epoch {epoch+1}/{epochs}")
                self.model.fit(X_train, X_train, epochs=1, batch_size=batch_size)

                # Calculate validation loss manually
                val_loss_epoch = self.model.evaluate(val_data, val_labels, batch_size=batch_size)
                val_loss.append(val_loss_epoch)

                print(f"Validation loss for epoch {epoch+1}: {val_loss_epoch}")

            return val_loss
        else:
            self.model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size)

# RUN BOTTOM FOR FINAL HYPERPARAMETERS

# Split the data into training and validation sets
# X_train, X_val = train_test_split(all_windows, test_size=0.2, random_state=42)

# # Apply patch-based masking to both training and validation data
# X_train_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_train])
# X_val_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_val])

# # Create the model
# input_shape = (window_size, 1)  # Shape of each window (500, 1)
# autoencoder = FullyConvolutionalMaskedAutoencoder(input_shape, num_layers=3, filters=[64, 32, 16])

# # Compile the model
# autoencoder.compile_model(optimizer='adam', loss='mean_squared_error')

# # Train the model with validation data
# autoencoder.fit(X_train_masked, epochs=1, batch_size=32, validation_data=(X_val_masked, X_val))

# Define the encoder model using the functional API
# encoder_input = autoencoder.model.input  # The input layer
# encoder_output = autoencoder.model.layers[sum([2 if i < autoencoder.num_layers - 1 else 1 for i in range(autoencoder.num_layers)]) - 1].output  # The last encoder layer's output

# encoder_model = tf.keras.Model(inputs=encoder_input, outputs=encoder_output)  # Create the encoder model

# # Define the path where you want to save the encoder
# os.makedirs(models_path, exist_ok=True)  # Ensure the directory exists
# path = os.path.join(models_path, 'encoder_model.keras')

# # Save the encoder model
# encoder_model.save(path)
# print(f'Saved encoder at {path}!')

In [5]:
from skopt import gp_minimize
from skopt.space import Integer, Real
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Define the search space
search_space = [
    Real(1e-5, 1e-2, "log-uniform", name='learning_rate'),
    Categorical([16, 32, 64], name='batch_size'),
    Integer(2, 5, name='num_layers'),
    Integer(16, 128, name='base_filters')  
]

# Create a function to build and compile the model with hyperparameters
def build_model(learning_rate, num_layers, base_filters):
    input_shape = (500, 1)  # Shape of each time series window

    # Define filter sizes: Increasing in encoder, decreasing in decoder
    filters = [base_filters * (2**i) for i in range(num_layers)]  # Encoder: 2x growth
    filters += filters[-2::-1]  # Decoder: Mirror decreasing pattern

    # Instantiate and compile the model using your custom class
    model = FullyConvolutionalMaskedAutoencoder(input_shape, num_layers=num_layers, filters=filters)
    model.compile_model(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mean_squared_error')

    return model


def objective(params):
    learning_rate, batch_size, num_layers, base_filters = params  # ✅ base_filters instead of filters_per_layer
    
    # Compute filter sizes
    filters = [base_filters * (2**i) for i in range(num_layers)]  # Encoder: 2x growth
    filters += filters[-2::-1]  # Decoder: Mirror decreasing pattern
    
    # Print the hyperparameters and filters for each layer
    print(f"Learning Rate: {learning_rate}, Batch Size: {batch_size}, Num Layers: {num_layers}, Base Filters: {base_filters}")
    print(f"Filters per layer: {filters}")  # Print filter sizes for each layer
    
    # Build the model
    model = build_model(learning_rate, num_layers, base_filters)
    
    # Split the data into training and validation sets
    X_train, X_val = train_test_split(all_windows, test_size=0.2, random_state=42)
    
    # Apply masking to the data
    X_train_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_train])
    X_val_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_val])
    
    # Train the model
    model.fit(X_train_masked, epochs=5, batch_size=batch_size, validation_data=(X_val_masked, X_val))
    
    # Get validation loss
    val_loss = model.model.history.history['val_loss'][-1]

    # Print the validation loss to the console
    print(f"Validation Loss: {val_loss}")
    
    # Append the results to a file
    with open("FCMAE_tuning.txt", "a") as file:
        file.write(f"Learning Rate: {learning_rate}, Batch Size: {batch_size}, Num Layers: {num_layers}, Base Filters: {base_filters}\n")
        file.write(f"Filters per layer: {filters}\n")  # Log filters for each layer
        file.write(f"Validation Loss: {val_loss}\n\n")
    
    return val_loss

# Run Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=15, random_state=42)

# Print the best hyperparameters and validation loss to the console
best_params = result.x
print(f"Best Hyperparameters: {best_params}")

with open("FCMAE_tuning.txt", "a") as file:
    file.write(f"Best Hyperparamters: {best_params}")

Learning Rate: 0.002452612631133679, Batch Size: 16, Num Layers: 4, Base Filters: 32
Filters per layer: [32, 64, 128, 256, 128, 64, 32]


NameError: name 'FullyConvolutionalMaskedAutoencoder' is not defined